In [53]:
import torch
from torch import nn as nn
from torch.nn import functional as F
import numpy as np

In [45]:
# Device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
# Hyperparameters
block_size = 8
batch_size = 4

In [54]:
with open("content.txt", 'r', encoding='utf-8') as f:
    for line in f:
        text = f.read()

characters = sorted(set(text))
vocab_size = len(characters)

In [28]:
# Creating a mapping from unique characters to indices
string_to_int = { char:index for index, char in enumerate(characters)}
int_to_string = { index:char for index, char in enumerate(characters)}

# Encoding the text
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda c : ''.join([int_to_string[i] for i in c])

# Turn into tensors 
data = torch.tensor(encode(text), dtype=torch.long)


In [52]:
split = int(len(data) * 0.8) # Hyperparameter

train_data = data[:split].to(device)
test_data = data[split:].to(device)

def get_batch(split):
    data = train_data if split == "train" else test_data
    start_index = torch.randint(len(data) - block_size, (batch_size,))
    print(start_index)
    x = torch.stack([data[idx:idx+block_size] for idx in start_index])
    y = torch.stack([data[idx+1:idx+block_size+1] for idx in start_index])
    x, y = x.to(device), y.to(device) # Make sure data is on the right device
    return x, y

x, y = get_batch("train")
print("inputs:")
print(x.shape)
print(x)
print("targets:")
print(y)


tensor([1591208,  213786,  902263,  209614])
inputs:
torch.Size([4, 8])
tensor([[49, 62,  1, 48, 51, 45, 53,  1],
        [43,  1, 46, 47,  1, 48, 73, 49],
        [60, 63, 53, 43, 46, 47,  1, 53],
        [ 1, 82,  1, 19, 53, 62, 43,  1]], device='cuda:0')
targets:
tensor([[62,  1, 48, 51, 45, 53,  1, 46],
        [ 1, 46, 47,  1, 48, 73, 49, 54],
        [63, 53, 43, 46, 47,  1, 53, 57],
        [82,  1, 19, 53, 62, 43,  1, 47]], device='cuda:0')


In [44]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context}, the target is {target}")

When input is tensor([22], device='cuda:0'), the target is 47
When input is tensor([22, 47], device='cuda:0'), the target is 56
When input is tensor([22, 47, 56], device='cuda:0'), the target is 1
When input is tensor([22, 47, 56,  1], device='cuda:0'), the target is 61
When input is tensor([22, 47, 56,  1, 61], device='cuda:0'), the target is 51
When input is tensor([22, 47, 56,  1, 61, 51], device='cuda:0'), the target is 49
When input is tensor([22, 47, 56,  1, 61, 51, 49], device='cuda:0'), the target is 56
When input is tensor([22, 47, 56,  1, 61, 51, 49, 56], device='cuda:0'), the target is 43


In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)


    def forward(self, index, targets): 
        logits = self.token_embedding_table(index)
        B, T, C = logits.shape # batch_size, time, vocab_size
        logits = logits.view(B*T, C) # Flatten the batch and time dimensions
        targets = targets.view(B*T) # Flatten the batch and time dimensions 
        loss = F.cross_entropy(logits, targets) # Calculate the loss 

        return logits, loss

    # Generate new content
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            # Make predictions
            logits, loss = self.forward(index)
            # Get the last time step
            logits = logits[:, -1, :] # (B, C)
            # Turn into probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # Sample from the distribution or take the most likely
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # Append to the sequence
            index = torch.cat([index, index_next], dim=-1) # (B, T+1)
        return index